# Look for B0 candidates

Import the required modules

In [1]:
import sys
import basf2 as b2
# Convenience modules
import modularAnalysis as ma
# Convenience functions to filter standard particles
import stdV0s
# To print statistics
from basf2 import statistics
# To handle variables
import variables.collections as vc
import variables.utils as vu
from variables import variables as vm 
# To tag the flavour of the reconstructed B0
import flavorTagger as ft
# To do vertex fitting
import vertex

Welcome to JupyROOT 6.20/04


Select the input file number from the examples to analyze and if we want to run the version with cuts (without background) or without cuts (with background)

In [2]:
filenumber = 1

## Create a Path and add a processig pipeline to it
This will be define the container of the series of steps we are going to perform to do our analysis. All steps will be added as "modules" to this path

In [3]:
main = b2.Path()

Now, we can start adding modules to the processing pipeline (i.e. to the Path)

### Load input data from mdst/udst file
We are going to add a module to the "main" path to indicate the input file containing all the events

In [4]:
# load input data from mdst/udst file
ma.inputMdstList(
    environmentType="default",
    filelist=[b2.find_file(f"starterkit/2021/1111540100_eph3_BGx0_{filenumber}.root", "examples")],
    path=main,
)

Now, we are going to add the steps that actually filter the data. Each filter is looking for a specific particle and each candidate (i.e. the result of every filter) is going to be stored in the ParticleList. The goal is to create filters and recontruct everything we need so, in the end, we can find B0 candidates.

### Load electrons, positrons.
First, we are going to look for electrons and positrons. That is, we are going "fill the particle list" with electron/positron candidates based on our filtering criteria (i.e. based on "cuts")

In [5]:
# fill final state particle lists
ma.fillParticleList(
    "e+:uncorrected",
    "electronID > 0.1 and dr < 0.5 and abs(dz) < 2 and thetaInCDCAcceptance",
    path=main,
)

### Load k_shorts
Then, we going to look for K_short candidates. The appropiate cuts and filters are already programmed and store in a convenience function called "stdKshorts" (which is in the module of convenience filters for Standard Particles: stdV0s).

In its inner workings, this function applies the recommended cuts (it looks for the K_S0 by looking for its product decays: K_S0 -> pi+ pi-. It looks for them using two different methods [V0 and ParticleCombiner] and then merges the results).

K_S0 candidates in a variable called K_S0:merged in the ParticleList

In [6]:
stdV0s.stdKshorts(path=main)

### Load Bremsstrahlung photons
Define the cuts that photons should pass in order to match the physics of Bremsstrahlung photons

In [7]:
vm.addAlias(
    "goodFWDGamma", "passesCut(clusterReg == 1 and clusterE > 0.075)"
)
vm.addAlias(
    "goodBRLGamma", "passesCut(clusterReg == 2 and clusterE > 0.05)"
)
vm.addAlias(
    "goodBWDGamma", "passesCut(clusterReg == 3 and clusterE > 0.1)"
)
vm.addAlias(
    "goodGamma", "passesCut(goodFWDGamma or goodBRLGamma or goodBWDGamma)"
)

True

Recover the Bremsstrahlung photons of the uncorrected positrons using the cuts we've just defined

In [8]:
ma.fillParticleList("gamma:brems", "goodGamma", path=main)
ma.correctBrems("e+:corrected", "e+:uncorrected", "gamma:brems", path=main)
vm.addAlias("isBremsCorrected", "extraInfo(bremsCorrected)")

True

### Reconstruct J/psi
Now, we are looking for J/psi candidates. There is no convenience function, but it is fairly easy to look for the J/psi by its products e+ e-. The syntax for the decayString is:

````
parent:label -> children
````
 
where "label" is an optional label to add to the variable in the ParticleList If no label is provided, then the variable is just called "parent".

The cut we are using is setting a mass minus nominal mass lower than 0.11

In [9]:
ma.reconstructDecay(
    "J/psi:ee -> e+:corrected e-:corrected ?addbrems",
    cut="dM < 0.11",
    path=main,
)

### Reconstruct B0

Up to this point, we have candidates for the J/psi and the K_S0 (stored in the variable K_S0:merged), we can use those them to create a new candidate for the B0 particle.

We are adding a cut to remove candidates that:

1. Have a beam-constrained energy greater than 5.2
2. $ |\Delta E| < 0.15$

In [10]:
ma.reconstructDecay(
    "B0 -> J/psi:ee K_S0:merged",
    cut="Mbc > 5.2 and abs(deltaE) < 0.15",
    path=main,
)

### Checking if it is a signal candidate or background (only available for MonteCarlo data)

Add MC matching for all particles of the decay chain and add the information whether the reconstructed B meson is a signal candidate to the ntuple. 

`matchMCTruth` stores the result in a variable called `isSignal`

In [11]:
# match reconstructed with MC particles
ma.matchMCTruth("B0", path=main)

### Build the ROE
Build the Rest of Event around the reconstructed B0 and add its variables to the ROOT file we want to create so we can use them later

To make sure this ROE is useful, we need to "clean it up" by cutting out background particles that don't really matter for our analysis. In this case, we'll perform the following cuts:

In [12]:
ma.buildRestOfEvent("B0", fillWithMostLikely=True, path=main)
# create a mask tuple (name, cut1, cut2, ...)
track_based_cuts = "thetaInCDCAcceptance and pt > 0.075 and dr < 5 and abs(dz) < 10"
ecl_based_cuts = "thetaInCDCAcceptance and E > 0.05"
roe_mask = ("my_mask", track_based_cuts, ecl_based_cuts)
# append the mask to the path
ma.appendROEMasks("B0", [roe_mask], path=main)

#### Flavour Tagging of the B0 candidate
The `flavorTagger` uses machine learning techniques to infer the flavour of a particle (in this case, the $B_0$ based on its ROE, so now that we have build the ROE it is a good time to add the flavor tagger.

In [13]:
# Specify the model to use
b2.conditions.prepend_globaltag("analysis_tools_release-04-02")
# Add the flavor tagger
ft.flavorTagger(
    particleLists=["B0"],
    path=main
)

[INFO] *** FLAVOR TAGGING ***
[INFO]  
[INFO]     Working directory is: ./FlavorTagging/TrainedMethods
[INFO]  
[INFO] EVENT LEVEL
[INFO] flavorTagger: Applying MVAExpert FlavorTagger_Belle2_B2nunubarBGx1EventLevelElectronFBDT.
[INFO] flavorTagger: Applying MVAExpert FlavorTagger_Belle2_B2nunubarBGx1EventLevelIntermediateElectronFBDT.
[INFO] flavorTagger: Applying MVAExpert FlavorTagger_Belle2_B2nunubarBGx1EventLevelMuonFBDT.
[INFO] flavorTagger: Applying MVAExpert FlavorTagger_Belle2_B2nunubarBGx1EventLevelIntermediateMuonFBDT.
[INFO] flavorTagger: Applying MVAExpert FlavorTagger_Belle2_B2nunubarBGx1EventLevelKinLeptonFBDT.
[INFO] flavorTagger: Applying MVAExpert FlavorTagger_Belle2_B2nunubarBGx1EventLevelIntermediateKinLeptonFBDT.
[INFO] flavorTagger: Applying MVAExpert FlavorTagger_Belle2_B2nunubarBGx1EventLevelKaonFBDT.
[INFO] flavorTagger: Applying MVAExpert FlavorTagger_Belle2_B2nunubarBGx1EventLevelSlowPionFBDT.
[INFO] flavorTagger: Applying MVAExpert FlavorTagger_Belle2_B2nunub

### Perform vertex fitting
We want to reconstruct the $B$ vertex, so we are going to approximate it with the $J/\Psi$ vertex because:

* We can't reconstruct the B vertex directly because its decay products ($J/\Psi$ and $K_S^0$) are not charged particles, so they don't leave a track and thus we can't identify it's vertex

* We could use the $K_S^0$ vertex, but it lives longer than the $J/\Psi$, so it can decay several centimeters away from where it was produced (the $B$ vertex) thus it is a bad approximation

In [14]:
vertex.kFit("J/psi:ee", conf_level=0.0, path=main)

### Perform vertex fitting on the other B meson
Since B mesons are produced in pairs, for every signal candidate we reconstruct, there will also be another (the "tag") which is not explicitly reconstructed. We weill get its position via a geometric fit over the ROE via `TagV`:

In [15]:
vertex.TagV("B0", constraintType="tube", path=main)

### Select best candidates
Usually we need to select the best candidates of all the results to perform our analysis, but in this case let's just add a random variable to rank them

In [16]:
# perform best candidate selection
b2.set_random_seed("Belle II StarterKit")
ma.rankByHighest("B0", variable="random", numBest=1, path=main)

[INFO] The random number seed is set to "Belle II StarterKit"


## Define variables to store

In [17]:
# Create list of variables to save into the output file
b_vars = []

standard_vars = vc.kinematics + vc.mc_kinematics + vc.mc_truth
b_vars += vc.deltae_mbc
b_vars += standard_vars

# ROE variables
roe_kinematics = ["roeE()", "roeM()", "roeP()", "roeMbc()", "roeDeltae()"]
roe_multiplicities = [
    "nROE_Charged()",
    "nROE_Photons()",
    "nROE_NeutralHadrons()",
]
b_vars += roe_kinematics + roe_multiplicities
# Let's also add a version of the ROE variables that includes the mask:
for roe_variable in roe_kinematics + roe_multiplicities:
    # e.g. instead of 'roeE()' (no mask) we want 'roeE(my_mask)'
    roe_variable_with_mask = roe_variable.replace("()", "(my_mask)")
    b_vars.append(roe_variable_with_mask)
    
# add variables corresponding to the flavor tagger
b_vars += ft.flavor_tagging

# add variables corresponding to the tag vertex
b_vars += vc.tag_vertex + vc.mc_tag_vertex

### Add other useful variables in the reconstruction of the B0 to store in the ROOT file

A good variable to start with is the beam-constrained mass Mbc, which will be stored in a tuple. This variable:

1. is only of interest for B0 candidates, so it will be one entry per candidate which is defined by the decayString

2. has to peak at the mass of the B0 (which will mean filters are indeed working and we have good candidates)

3. will be stored in a file called "Bd2JpsiKS.root" in a tree called "tree"

As the data comes from a MC simulation, we know if the candidates are correct or not with the `isSignal` variable, so we are going to store it as well.

We will store other variables so, for the process:

$$ B_0 \rightarrow (J/\Psi \rightarrow e^+ e^-) (K_s^0 \rightarrow \pi^+ \pi^-) $$

We will end up storing:

* For all particles : standard variables (reconstructed kinematics in the lab frame `vc.kinematics`, truth kinematics in the lab frame `vc.mc_kinematics`) and the kinematics in the CMS `useCMSFrame(kinematics)`.
* For the $B_0$: Mbc and deltaE `vc.deltae_mbc`
* For the resonances $J/\Psi$ and $K_S^0$: the invariant mass `vc.inv_mass`
* For the final state charged particles (electron, positron and pions): PID and track data

In [18]:
# add PID and track variables for all charged particles in the final
# states
fs_vars = vc.pid + vc.track + vc.track_hits + standard_vars
b_vars += vu.create_aliases_for_selected(
    fs_vars + ["isBremsCorrected"],
    "B0 -> [J/psi -> ^e+ ^e-] K_S0",
    prefix=["ep", "em"],
)
b_vars += vu.create_aliases_for_selected(
    fs_vars, "B0 -> J/psi [K_S0 -> ^pi+ ^pi-]", prefix=["pip", "pim"]
)

# add the invariant mass of the intermediate resonances to the ntuple
jpsi_ks_vars = vc.inv_mass + standard_vars
# add the vertex reconstruction variables
jpsi_ks_vars += vc.vertex + vc.mc_vertex
b_vars += vu.create_aliases_for_selected(jpsi_ks_vars, "B0 -> ^J/psi ^K_S0")

And we are creating a variable to store the invariant mass of the $J/\Psi$ meson using the uncorrected momenta of the leptons. We'll later compare it against the corrected mass

In [19]:
# will give us the invariant mass of the first daughter of the
# first daughter, and the first daughter of the second daughter. 
# Add the J/Psi mass calculated with uncorrected electrons:
vm.addAlias(
    "Jpsi_M_uncorrected", "daughter(0, daughterCombination(M,0:0,1:0))"
)
b_vars += ["Jpsi_M_uncorrected"]

Add kinematics variables in the Center of Mass frame for all particles

In [20]:
# Add the kinematics in the CMS frame for all particles
cmskinematics = vu.create_aliases(
    vc.kinematics, "useCMSFrame({variable})", "CMS"
)
b_vars += vu.create_aliases_for_selected(
    cmskinematics, "^B0 -> [^J/psi -> ^e+ ^e-] [^K_S0 -> ^pi+ ^pi-]"
)

Add Brems variables

In [21]:
vm.addAlias(
    "withBremsCorrection",
    "passesCut(passesCut(ep_isBremsCorrected == 1) or passesCut(em_isBremsCorrected == 1))",
)
b_vars += ["withBremsCorrection"]

### Store the results in a ROOT file
Finally, we add the module `variablesToNtuple` to store all the given variables we constructed `b_vars` in a ROOT file, but first let's inspect all the variables we have added

In [22]:
from variables import variables as vm
vm.printAliases()

[INFO] =====================================
[INFO] The following aliases are registered:
[INFO] CMS_E                                --> useCMSFrame(E)
[INFO] CMS_p                                --> useCMSFrame(p)
[INFO] CMS_pt                               --> useCMSFrame(pt)
[INFO] CMS_px                               --> useCMSFrame(px)
[INFO] CMS_py                               --> useCMSFrame(py)
[INFO] CMS_pz                               --> useCMSFrame(pz)
[INFO] FANN_qrCombined                      --> qrOutput(FANN)
[INFO] FBDT_qrCombined                      --> qrOutput(FBDT)
[INFO] Jpsi_CMS_E                           --> daughter(0,CMS_E)
[INFO] Jpsi_CMS_p                           --> daughter(0,CMS_p)
[INFO] Jpsi_CMS_pt                          --> daughter(0,CMS_pt)
[INFO] Jpsi_CMS_px                          --> daughter(0,CMS_px)
[INFO] Jpsi_CMS_py                          --> daughter(0,CMS_py)
[INFO] Jpsi_CMS_pz                          --> daughter(0,CMS_pz)
[I

[INFO] K_S0_mcProductionVertexZ             --> daughter(1,mcProductionVertexZ)
[INFO] K_S0_p                               --> daughter(1,p)
[INFO] K_S0_pi_0_CMS_E                      --> daughter(1,daughter(0,CMS_E))
[INFO] K_S0_pi_0_CMS_p                      --> daughter(1,daughter(0,CMS_p))
[INFO] K_S0_pi_0_CMS_pt                     --> daughter(1,daughter(0,CMS_pt))
[INFO] K_S0_pi_0_CMS_px                     --> daughter(1,daughter(0,CMS_px))
[INFO] K_S0_pi_0_CMS_py                     --> daughter(1,daughter(0,CMS_py))
[INFO] K_S0_pi_0_CMS_pz                     --> daughter(1,daughter(0,CMS_pz))
[INFO] K_S0_pi_1_CMS_E                      --> daughter(1,daughter(1,CMS_E))
[INFO] K_S0_pi_1_CMS_p                      --> daughter(1,daughter(1,CMS_p))
[INFO] K_S0_pi_1_CMS_pt                     --> daughter(1,daughter(1,CMS_pt))
[INFO] K_S0_pi_1_CMS_px                     --> daughter(1,daughter(1,CMS_px))
[INFO] K_S0_pi_1_CMS_py                     --> daughter(1,daughter(1,CM

[INFO] ep_pionID                            --> daughter(0,daughter(0,pionID))
[INFO] ep_protonID                          --> daughter(0,daughter(0,protonID))
[INFO] ep_pt                                --> daughter(0,daughter(0,pt))
[INFO] ep_px                                --> daughter(0,daughter(0,px))
[INFO] ep_py                                --> daughter(0,daughter(0,py))
[INFO] ep_pz                                --> daughter(0,daughter(0,pz))
[INFO] ep_z0                                --> daughter(0,daughter(0,z0))
[INFO] goodBRLGamma                         --> passesCut(clusterReg == 2 and clusterE > 0.05)
[INFO] goodBWDGamma                         --> passesCut(clusterReg == 3 and clusterE > 0.1)
[INFO] goodFWDGamma                         --> passesCut(clusterReg == 1 and clusterE > 0.075)
[INFO] goodGamma                            --> passesCut(goodFWDGamma or goodBRLGamma or goodBWDGamma)
[INFO] hasTrueTargetElectron                --> hasTrueTargets(Electron)
[IN

[INFO] pip_muonID                           --> daughter(1,daughter(0,muonID))
[INFO] pip_nCDCHits                         --> daughter(1,daughter(0,nCDCHits))
[INFO] pip_nPXDHits                         --> daughter(1,daughter(0,nPXDHits))
[INFO] pip_nSVDHits                         --> daughter(1,daughter(0,nSVDHits))
[INFO] pip_nVXDHits                         --> daughter(1,daughter(0,nVXDHits))
[INFO] pip_ndf                              --> daughter(1,daughter(0,ndf))
[INFO] pip_p                                --> daughter(1,daughter(0,p))
[INFO] pip_pValue                           --> daughter(1,daughter(0,pValue))
[INFO] pip_pionID                           --> daughter(1,daughter(0,pionID))
[INFO] pip_protonID                         --> daughter(1,daughter(0,protonID))
[INFO] pip_pt                               --> daughter(1,daughter(0,pt))
[INFO] pip_px                               --> daughter(1,daughter(0,px))
[INFO] pip_py                               --> daughter(1

Now, we are good to go!

In [23]:
filename = f"Bd2JpsiKS_{filenumber}.root"
# store results in a root file
ma.variablesToNtuple(
    decayString="B0", variables=b_vars,
    filename=filename,
    treename="tree",
    path=main
)

## Perform processing

In [24]:
# Start the event loop (actually start processing things)
print(f"Storing results in {filename}...")
b2.process(main)

Storing results in Bd2JpsiKS_1.root...
